In [1]:
import dwave
import numpy as np
import openml
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import plotly.express as px
from plotly.subplots import make_subplots
import dimod
from dwave.system import LeapHybridCQMSampler

# dataset = openml.datasets.get_dataset(312)

clf = RandomForestClassifier()

In [3]:
df = pd.read_csv("Dataset-stock.csv")

In [4]:
df.head()

,Date,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,...,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30,Price Change,Class
0,2021-02-09,136.009995,137.880005,135.850006,136.201999,136.125000,134.943501,133.860334,56.309437,38.978440,...,100.0,80.0,73.333333,53.333333,55.0,85.0,36.666667,80.000000,0.619995,0
1,2021-02-10,135.389999,136.990005,134.399994,136.491998,135.458000,135.273000,133.817000,47.291372,34.811803,...,50.0,90.0,80.000000,60.000000,60.0,90.0,40.000000,83.333333,0.259995,0
2,2021-02-11,135.130005,136.389999,133.770004,136.039999,135.262001,135.485001,133.825667,42.237979,33.064527,...,60.0,100.0,86.666667,66.666667,65.0,95.0,43.333333,86.666667,-0.239990,1
3,2021-02-12,135.369995,135.529999,133.690002,135.762000,135.603000,135.808000,133.880667,32.659620,36.621645,...,70.0,80.0,93.333333,73.333333,70.0,100.0,46.666667,90.000000,2.179993,0
4,2021-02-16,133.190002,136.009995,132.789993,135.017999,135.508000,136.110500,133.897334,28.975106,21.149727,...,80.0,10.0,100.000000,80.000000,75.0,100.0,50.000000,93.333333,2.350006,0


In [5]:
df.drop('Price Change',axis=1,inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           473 non-null    object 
 1   Close          473 non-null    float64
 2   High           473 non-null    float64
 3   Low            473 non-null    float64
 4   SMA5           473 non-null    float64
 5   SMA10          473 non-null    float64
 6   SMA20          473 non-null    float64
 7   SMA30          473 non-null    float64
 8   RSI            473 non-null    float64
 9   %K             473 non-null    float64
 10  %D             473 non-null    float64
 11  ATR            473 non-null    float64
 12  ADX            473 non-null    float64
 13  EMA5           473 non-null    float64
 14  EMA10          473 non-null    float64
 15  EMA20          473 non-null    float64
 16  EMA30          473 non-null    float64
 17  Aroon Up 5     473 non-null    float64
 18  Aroon Down

In [7]:
# from sklearn.datasets import load_breast_cancer
from dwave.plugins.sklearn import SelectFromQuadraticModel

X = df.drop(['Class','Date'], axis=1)
y = df['Class']


X.head()

,Close,High,Low,SMA5,SMA10,SMA20,SMA30,RSI,%K,%D,...,Aroon Up 5,Aroon Down 5,Aroon Up 10,Aroon Down 10,Aroon Up 15,Aroon Down 15,Aroon Up 20,Aroon Down 20,Aroon Up 30,Aroon Down 30
0,136.009995,137.880005,135.850006,136.201999,136.125000,134.943501,133.860334,56.309437,38.978440,47.875752,...,80.0,100.0,100.0,80.0,73.333333,53.333333,55.0,85.0,36.666667,80.000000
1,135.389999,136.990005,134.399994,136.491998,135.458000,135.273000,133.817000,47.291372,34.811803,41.444801,...,100.0,20.0,50.0,90.0,80.000000,60.000000,60.0,90.0,40.000000,83.333333
2,135.130005,136.389999,133.770004,136.039999,135.262001,135.485001,133.825667,42.237979,33.064527,35.618257,...,80.0,20.0,60.0,100.0,86.666667,66.666667,65.0,95.0,43.333333,86.666667
3,135.369995,135.529999,133.690002,135.762000,135.603000,135.808000,133.880667,32.659620,36.621645,34.832658,...,100.0,40.0,70.0,80.0,93.333333,73.333333,70.0,100.0,46.666667,90.000000
4,133.190002,136.009995,132.789993,135.017999,135.508000,136.110500,133.897334,28.975106,21.149727,30.278633,...,100.0,20.0,80.0,10.0,100.000000,80.000000,75.0,100.0,50.000000,93.333333


In [ ]:
X_new = SelectFromQuadraticModel(num_features=5).fit_transform(X, y)


In [9]:
print(X_new.shape)

X_new

(473, 5)


array([[ 21.27763009, 135.49720544, 100.        , 100.        ,
         80.        ],
       [ 22.88460878, 135.48699534,  20.        ,  50.        ,
         83.33333333],
       [ 24.19728024, 135.45299625,  20.        ,  60.        ,
         86.66666667],
       ...,
       [ 16.67042996, 141.37614465,  20.        ,  60.        ,
          3.33333333],
       [ 17.58062904, 140.81174963,  40.        ,  70.        ,
          6.66666667],
       [ 18.6449141 , 139.99443973,  20.        ,  80.        ,
          3.33333333]])

In [10]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

In [11]:
def train_test_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create a list to store the results
    results = []
    
    # Models
    models = {
        'SVM': SVC(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'KNN': KNeighborsClassifier(),
        'Logistic Regression': LogisticRegression(max_iter=1500),
        'Naive Bayes': GaussianNB(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'XGBoost': xgb.XGBClassifier()
    }
    
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate accuracy and F1 score
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Store the results as a dictionary
        results.append({'Model': model_name, 'Accuracy': accuracy, 'F1 Score': f1})
    
    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)
    
    return results_df

In [13]:

# Train and test the models
results = train_test_models(X_new, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.589474,0.613861
1,Decision Tree,0.526316,0.545455
2,Random Forest,0.589474,0.580645
3,KNN,0.442105,0.475248
4,Logistic Regression,0.547368,0.598131
5,Naive Bayes,0.494737,0.538462
6,Gradient Boosting,0.547368,0.527473
7,XGBoost,0.589474,0.580645


In [14]:
X_new3 = SelectFromQuadraticModel(num_features=3).fit_transform(X, y)


/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [15]:
# Train and test the models
results = train_test_models(X_new3, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.421053,0.444444
1,Decision Tree,0.547368,0.505747
2,Random Forest,0.536842,0.531915
3,KNN,0.505263,0.459770
4,Logistic Regression,0.442105,0.495238
5,Naive Bayes,0.463158,0.523364
6,Gradient Boosting,0.505263,0.505263
7,XGBoost,0.484211,0.436782


In [16]:
X_new8 = SelectFromQuadraticModel(num_features=8).fit_transform(X, y)

/home/naman/Documents/RASSE/ocean/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SelectFromQuadraticModel was fitted without feature names
  warnings.warn(


In [17]:
# Train and test the models
results = train_test_models(X_new8, y)

# Print the results
results.head(10)

,Model,Accuracy,F1 Score
0,SVM,0.568421,0.601942
1,Decision Tree,0.526316,0.505495
2,Random Forest,0.568421,0.577320
3,KNN,0.536842,0.551020
4,Logistic Regression,0.568421,0.601942
5,Naive Bayes,0.515789,0.566038
6,Gradient Boosting,0.484211,0.473118
7,XGBoost,0.473684,0.489796


In [18]:
num_cols = X_new.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa5 = pd.DataFrame(data=X_new, columns=col_names)

# Save the DataFrame to a CSV file
qa5.to_csv('QA-5-features.csv', index=False)

In [20]:
num_cols = X_new3.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa3 = pd.DataFrame(data=X_new3, columns=col_names)

# Save the DataFrame to a CSV file
qa3.to_csv('QA-3-features.csv', index=False)

In [ ]:
num_cols = X_new8.shape[1]
col_names = [f'col{i+1}' for i in range(num_cols)]
qa8 = pd.DataFrame(data=X_new8, columns=col_names)

# Save the DataFrame to a CSV file
qa8.to_csv('QA-8-features.csv', index=False)